## General Workflow

The code in this Jupyter notebook is designed to identify inactive users of ArcGIS Pro within an organization. The workflow is as follows:

1. Import necessary libraries.
2. Prompt the user to enter their ArcGIS Online credentials.
3. Connect to ArcGIS Online using the provided credentials.
4. Retrieve all ArcGIS Pro licenses and the associated usernames.
5. Identify users who have not logged in within the past three months.
6. Check if these inactive users have published any items within the past three months.

---

**Cell 1: Importing Libraries**

---


In [ ]:
from arcgis.gis import GIS

import time
from datetime import datetime

import getpass

---

**Cell 2: User Authentication**

In this cell, we prompt the user to enter their ArcGIS Online credentials. These credentials are used to establish a connection to ArcGIS Online.

---


In [ ]:
username = input("Enter your username: ")
password = getpass.getpass("Enter your password: ")

# Connect to ArcGIS Online
gis = GIS("https://www.arcgis.com", username, password)


---

**Cell 3: Retrieving ArcGIS Pro Licenses**

This cell is responsible for retrieving all ArcGIS Pro licenses and the associated usernames. This information is crucial for identifying inactive users in the subsequent steps.

---

In [ ]:
# Get the License Manager
license_manager = gis.admin.license

# Get the ArcGIS Pro License
pro_license = license_manager.get('ArcGIS Pro')

# Get all the licenses
all_licenses = pro_license.all()

# Get the usernames of users who didn't login in the past 3 months
pro_usernames = [user['username'] for user in all_licenses]

print(pro_usernames)


---

**Cell 4: Identifying Inactive Users**

In this cell, we identify users who have not logged in within the past three months. This is done by comparing the current timestamp with the last login timestamp of each user. Users who have not logged in within this period are considered inactive.

---

In [ ]:
# Get the current timestamp in milliseconds
current_timestamp = int(time.time() * 1000)

# Get the timestamp for 3 months ago
three_months_ago = current_timestamp - (3 * 30 * 24 * 60 * 60 * 1000)

user_last_login = {}

# Iterate over the pro_usernames list
for username in pro_usernames:
    # Get the user
    user = gis.users.get(username)
    
    # If the user exists and didn't login in the past 3 months or never logged in, add their username and last login date to the dictionary
    if user and (user.lastLogin < three_months_ago or user.lastLogin == -1):
        # Convert the lastLogin timestamp to a datetime object if the user has logged in before
        last_login_date = datetime.fromtimestamp(user.lastLogin / 1000) if user.lastLogin != -1 else None
        user_last_login[username] = last_login_date
print(user_last_login)

---

**Cell 5: Checking Inactive Users' Activity**

In this final cell, we check if the identified inactive users have published any items within the past three months. This is an additional measure to ensure that the users are indeed inactive, as they might not have logged in but could have been active through API calls or other means.

<div style="color: red;">Disclaimer: This piece of code may take a couple of minutes to execute, depending on the size of the organization and the number of items there.</div>

---

In [ ]:
non_publishing_usernames = []

# Iterate over the pro_usernames list
for username in pro_usernames:
    user = gis.users.get(username)
    
    # Get the user's folders
    folders = user.folders
    folders.append(None)  # Add None to check the root folder
    
    # Initialize a flag to indicate whether the user has published any items in the past 3 months
    has_published_recently = False
    
    # Iterate over the folders
    for folder in folders:
        # Get the items in the folder
        items = user.items(folder=folder, max_items=100)
        
        # Check if any item was created in the past 3 months
        if any(item['created'] > three_months_ago for item in items):
            has_published_recently = True
            break
    
    # If the user hasn't published any items in the past 3 months, add their username to the list
    if not has_published_recently:
        non_publishing_usernames.append(username)

print(non_publishing_usernames)